In [ ]:
import numpy as np
import pandas as pd
import datetime
import copy
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
from tensorflow import keras
from keras.layers import TimeDistributed, Conv1D, MaxPool1D, Flatten, LSTM, Dense, AveragePooling1D, Dropout, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.regularizers import l2
import sklearn.metrics as metrics
from pyarrow import feather

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1, 20, NUM_FEATURES, 1)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,20)[:,-1]
   return X_train, y_train
def DoiNhan(label: np.array):
  for i in range(len(label)):
    if label[i] == 14 or label[i] == 16 or label[i] == 13:
      label[i] = 2
    elif label[i] == 11 or label[i] == 5 or label[i] == 10:
      label[i] = 1
    elif label[i] == 4 or label[i] == 8 or label[i] == 9 or label[i] == 1 or label[i] == 7:
      label[i] = 0

In [ ]:
NUM_EPOCHS = 200
BATCH_SIZE = 32
client_lr = 3e-4

In [ ]:
def train_CNN(X, y, X_test):
    model = Sequential()
    model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='Same',
                     activation='relu', input_shape=(20,128,1)))
    model.add(Conv2D(filters=64 , kernel_size=(5, 5), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=client_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    model.fit(X, y, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, shuffle=True)
    predictions = model.predict(X_test, verbose=2, batch_size=BATCH_SIZE)
    y_pred = np.argmax(predictions, axis=-1)

    return y_pred

In [ ]:
def Training_Step():
  path = '/content/drive/MyDrive/Fixed_data/'
  test_name = 'd1test'
  train_name ='d1train'
  BATCH_SIZE = 32

  diff_distribution = feather.read_feather(path + 'domain2.feather')
  diff, diff_label = data_processing(diff_distribution, 128)
  DoiNhan(diff_label)

  for i in range(7, 11):
    print('Start training model CNN Ds-Dt with ' + str(i) + '% data train and ' + str(100 - i) + '% data for test')
    same_distribution = feather.read_feather(path + train_name + str(i))
    test_data = feather.read_feather(path + test_name + str(i))

    same, same_label = data_processing(same_distribution, 128)
    DoiNhan(same_label)

    X_train, y_train = np.concatenate([diff, same], axis=0), np.concatenate([diff_label, same_label])
    X_test, y_test = data_processing(test_data, 128)
    DoiNhan(y_test)

    y_pred = train_CNN(X_train, y_train, X_test)
    print(metrics.classification_report(y_test, y_pred))

In [ ]:
model = Training_Step()

NameError: name 'Training_Step' is not defined